In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import pandas as pd
tf.config.set_soft_device_placement(False)
tf.debugging.set_log_device_placement(True)
import numpy as np
from ampligraph.datasets import load_fb15k_237, load_yago3_10
from ampligraph.evaluation.protocol import create_mappings, to_idx

from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score
import time
print(tf.__version__)
assert(tf.__version__.startswith('2.4'))

from ampligraph.datasets import load_fb15k_237, load_fb13, load_fb15k, load_wn11, load_wn18, load_wn18rr, load_yago3_10
from ampligraph.latent_features import ScoringBasedEmbeddingModel

2.4.0-dev20201007


In [2]:
#loss function
def nll(scores_pos, scores_neg):
    neg_exp = tf.exp(scores_neg)
    pos_exp = tf.exp(scores_pos)
    softmax_score = pos_exp / (tf.reduce_sum(neg_exp, axis=0) + pos_exp)

    loss = -tf.math.log(softmax_score)
    return loss

# Train/eval without partition

In [3]:

optim = tf.optimizers.Adagrad(learning_rate=0.1)
# optim = 'adam'

# loss = nll
# loss = 'self_adversarial'
from ampligraph.latent_features.loss_functions import SelfAdversarialLoss
loss = SelfAdversarialLoss({'margin': 0.1, 'alpha': 5, 'reduction': 'sum'})

model = ScoringBasedEmbeddingModel(eta=50, 
                                     k=300,
                                     scoring_type='TransE')



model.compile(optimizer=optim, loss=loss)

from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('./chkpt1', monitor='loss', verbose=1, save_best_only=True, mode='min')

start = time.time()
model.fit('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
             batch_size=10000,
             epochs=10,
             validation_freq=5,
             validation_batch_size=100,
             validation_data = '/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
         callbacks=[checkpoint])

end = time.time()
print('Time taken:', end-start)

Epoch 1/10


/home/spai/.conda/envs/tf_nightly/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("PartitionedCall_2:1", shape=(None,), dtype=int32), values=Tensor("PartitionedCall_2:0", shape=(None, 300), dtype=float32), dense_shape=Tensor("PartitionedCall_2:2", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/home/spai/.conda/envs/tf_nightly/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("PartitionedCall_2:4", shape=(None,), dtype=int32), values=Tensor("PartitionedCall_2:3", shape=(None, 300), dtype=float32), dense_shape=Tensor("PartitionedCall_2:5", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may 

     28/Unknown - 2s 72ms/step - loss: 1.4029
Epoch 00001: loss improved from inf to 1.40221, saving model to ./chkpt1
29/29 [==============================] - 2s 71ms/step - loss: 1.4022
Epoch 2/10
28/29 [===========================>..] - ETA: 0s - loss: 1.3488
Epoch 00002: loss improved from 1.40221 to 1.34860, saving model to ./chkpt1
29/29 [==============================] - 1s 37ms/step - loss: 1.3486
Epoch 3/10
28/29 [===========================>..] - ETA: 0s - loss: 1.3259
Epoch 00003: loss improved from 1.34860 to 1.32573, saving model to ./chkpt1
29/29 [==============================] - 1s 38ms/step - loss: 1.3257
Epoch 4/10
28/29 [===========================>..] - ETA: 0s - loss: 1.3124
Epoch 00004: loss improved from 1.32573 to 1.31234, saving model to ./chkpt1
29/29 [==============================] - 1s 37ms/step - loss: 1.3123
Epoch 5/10
28/29 [===========================>..] - ETA: 0s - loss: 1.3033
9 triples containing invalid keys skipped!
177/177 [======================

In [4]:

start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
         use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
206/206 [==============================] - 54s 262ms/step
Time taken: 54.01689267158508


(265.7934484783247,
 0.18948653867548204,
 0.11143458264018005,
 0.3431108719052745)

## Load Checkpoint and evaluate

In [5]:
start = time.time()
loaded_model = ScoringBasedEmbeddingModel(eta=50, 
                                     k=300, 
                                     scoring_type='TransE')
loaded_model.load_weights('./chkpt1')
ranks = loaded_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
         use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
206/206 [==============================] - 54s 262ms/step
Time taken: 54.01914381980896


(265.7934484783247,
 0.18948653867548204,
 0.11143458264018005,
 0.3431108719052745)

# Training/eval with partition 

## Training with RandomEdges partitioner

In [6]:
from ampligraph.datasets import DummyBackend, SQLiteAdapter
from ampligraph.datasets import GraphDataLoader
from ampligraph.datasets.graph_partitioner import PARTITION_ALGO_REGISTRY
dataset_loader = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt', 
                                 backend=SQLiteAdapter,
                                    batch_size=1000, 
                                    dataset_type='train', 
                                    epochs=10,
                                    use_indexer=True)

# Choose the partitioner 
partitioner = PARTITION_ALGO_REGISTRY.get('RandomEdges')(dataset_loader, k=3)

_split: memory before: 0.0Bytes, after: 12.888MB, consumed: 12.888MB; exec time: 31.396s


In [7]:
optim = tf.optimizers.Adam(learning_rate=0.001, amsgrad=True)

partitioned_model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     scoring_type='TransE')
partitioned_model.compile(optimizer=optim, loss=nll)

In [8]:

start = time.time()
partitioned_model.fit(partitioner,
                     batch_size=1000, use_partitioning=True,
                     epochs=10)
print((time.time()-start))

Epoch 1/10
274/274 [==============================] - 16s 59ms/step - loss: 1.5947
Epoch 2/10
274/274 [==============================] - 15s 55ms/step - loss: 1.4213
Epoch 3/10
274/274 [==============================] - 15s 56ms/step - loss: 1.2638
Epoch 4/10
274/274 [==============================] - 15s 55ms/step - loss: 1.1327
Epoch 5/10
274/274 [==============================] - 15s 56ms/step - loss: 1.0269
Epoch 6/10
274/274 [==============================] - 15s 55ms/step - loss: 0.9414
Epoch 7/10
274/274 [==============================] - 15s 55ms/step - loss: 0.8716
Epoch 8/10
274/274 [==============================] - 15s 55ms/step - loss: 0.8139
Epoch 9/10
274/274 [==============================] - 15s 55ms/step - loss: 0.7652
Epoch 10/10
274/274 [==============================] - 15s 55ms/step - loss: 0.7237
164.69696426391602


In [9]:

dataset_loader_test = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                        backend=DummyBackend,
                                        batch_size=400, 
                                        dataset_type='test', 
                                        epochs=1,
                                        use_indexer=partitioned_model.data_handler.get_mapper())

start = time.time()
ranks = partitioned_model.evaluate(dataset_loader_test, 
                                   batch_size=400)
end = time.time()
print('Time taken:', end-start)

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!
53/53 [==============================] - 128s 2s/step
Time taken: 127.85662364959717


(488.4006996770721, 0.09720079053038014, 0.0, 0.2657305020060671)

## Save and Load

In [10]:
partitioned_model.save_weights('./best_model')

In [11]:
loaded_part_model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     scoring_type='TransE')

loaded_part_model.load_weights('./best_model')

In [12]:

dataset_loader_test = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                        backend=DummyBackend,
                                        batch_size=400, 
                                        dataset_type='test', 
                                        epochs=1,
                                        use_indexer=loaded_part_model.data_indexer)

start = time.time()
ranks = loaded_part_model.evaluate(dataset_loader_test, 
                                   batch_size=400)
end = time.time()
print('Time taken:', end-start)

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!
53/53 [==============================] - 131s 2s/step
Time taken: 130.72745776176453


(488.4006996770721, 0.09720079053038014, 0.0, 0.2657305020060671)

# Training/eval with partition (default Partitioning Approach)

In [13]:
from ampligraph.datasets import DummyBackend, SQLiteAdapter
from ampligraph.datasets import GraphDataLoader
from ampligraph.datasets.graph_partitioner import PARTITION_ALGO_REGISTRY


In [14]:
optim = tf.optimizers.Adam(learning_rate=0.001, amsgrad=True)

partitioned_model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     scoring_type='TransE')
partitioned_model.compile(optimizer=optim, loss=nll)


start = time.time()
partitioned_model.fit('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                     batch_size=1000, use_partitioning=True,
                     epochs=10)
print((time.time()-start))

_split: memory before: 102.71MB, after: 115.42MB, consumed: 12.706MB; exec time: 85.124s
Epoch 1/10
277/277 [==============================] - 9s 34ms/step - loss: 1.7309
Epoch 2/10
277/277 [==============================] - 9s 32ms/step - loss: 1.6420
Epoch 3/10
277/277 [==============================] - 9s 32ms/step - loss: 1.5553
Epoch 4/10
277/277 [==============================] - 9s 33ms/step - loss: 1.4762
Epoch 5/10
277/277 [==============================] - 9s 33ms/step - loss: 1.4055
Epoch 6/10
277/277 [==============================] - 9s 32ms/step - loss: 1.3427
Epoch 7/10
277/277 [==============================] - 9s 32ms/step - loss: 1.2868
Epoch 8/10
277/277 [==============================] - 9s 33ms/step - loss: 1.2369
Epoch 9/10
277/277 [==============================] - 9s 33ms/step - loss: 1.1923
Epoch 10/10
277/277 [==============================] - 9s 32ms/step - loss: 1.1523
183.76288604736328


In [15]:


start = time.time()
ranks = partitioned_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                   batch_size=400)
end = time.time()
print('Time taken:', end-start)

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!
53/53 [==============================] - 131s 2s/step
Time taken: 130.80173540115356


(769.9404295919365, 0.08808123789332144, 0.0, 0.23911341618553675)

## Filtered evaluation

In [16]:
start = time.time()
ranks = partitioned_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=400,
                       corrupt_side='s,o',
                        use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                              'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                              'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
53/53 [==============================] - 296s 6s/step
Time taken: 295.847562789917


(644.7648742538408,
 0.19486104781589067,
 0.13406399843428907,
 0.31321557882375967)